# Chroma

[![Index](https://img.shields.io/badge/Index-blue)](../index.ipynb)
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/digillia/Digillia-Colab/blob/main/tools/chroma.ipynb)

> <span style='color:red'>ATTENTION: notebook en cours d'élaboration!</span>

- https://www.trychroma.com/
- https://github.com/chroma-core/chroma

In [1]:
from IPython import get_ipython

# Supprimer les commentaires pour installer

# À installer dans tous les cas pour Google Colab
if 'google.colab' in str(get_ipython()):
    !pip3 install -q -U chromadb

## Création d'une Base Vectorielle Chroma

In [ ]:
import chromadb

# Exécuter chroma en mémoire
client = chromadb.Client()

# Création d'une collection
collection = client.create_collection("documents")


collection.add(
    documents=["This is document1", "This is document2"],
    metadatas=[{"source": "notion"}, {"source": "google-docs"}],
    ids=["doc1", "doc2"],
)



In [ ]:
# Query/search 2 most similar results. You can also .get by id
results = collection.query(
    query_texts=["This is a query document"],
    n_results=2,
    # where={"metadata_field": "is_equal_to_this"}, # optional filter
    # where_document={"$contains":"search_string"}  # optional filter
)